<a href="https://colab.research.google.com/github/eriksali/DNN_2023_NLP/blob/main/NLP04_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSI 5900: Lecture 04-05 Code Examples

Prof. Steven Wilson, Oakland University

## Language Identification with Naive Bayes

### Download the data

In [ ]:
! pip install datasets
from datasets import load_dataset
dataset = load_dataset("papluca/language-identification")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 6.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/papluca___csv/papluca--language-identification-ad5bdc8c9b1a4985/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 70000
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['labels', 'text'],
        num_rows: 10000
    })
})

In [ ]:
dataset['train'][0]

{'labels': 'pt',
 'text': 'os chefes de defesa da estónia, letónia, lituânia, alemanha, itália, espanha e eslováquia assinarão o acordo para fornecer pessoal e financiamento para o centro.'}

In [ ]:
# subsample a few langs to make it more manageable to train during class

from tqdm import tqdm

langs = set(['es','en','pt'])

small_dataset = {}

for split in ['train','validation','test']:
    small_dataset[split] = {'labels':[], 'text':[]}
    for item in tqdm(dataset[split]):
        lang =  item['labels']
        if lang in langs:
            small_dataset[split]['labels'].append(lang)
            small_dataset[split]['text'].append(item['text'])

print()
print(len(small_dataset['train']['text']))
print(len(small_dataset['test']['text']))



  0%|          | 0/70000 [00:00<?, ?it/s]

  3%|▎         | 2086/70000 [00:00<00:03, 20855.89it/s]

  7%|▋         | 4976/70000 [00:00<00:02, 25584.77it/s]

 11%|█         | 7535/70000 [00:00<00:03, 20619.68it/s]

 15%|█▍        | 10383/70000 [00:00<00:02, 23326.20it/s]

 19%|█▉        | 13303/70000 [00:00<00:02, 25262.06it/s]

 23%|██▎       | 15909/70000 [00:00<00:02, 20645.85it/s]

 26%|██▌       | 18139/70000 [00:00<00:02, 19343.62it/s]

 29%|██▉       | 20187/70000 [00:00<00:02, 18747.14it/s]

 32%|███▏      | 22357/70000 [00:01<00:02, 19521.15it/s]

 35%|███▍      | 24374/70000 [00:01<00:02, 19655.24it/s]

 39%|███▉      | 27466/70000 [00:01<00:01, 22809.49it/s]

 43%|████▎     | 30261/70000 [00:01<00:01, 24276.66it/s]

 47%|████▋     | 32741/70000 [00:01<00:01, 23879.38it/s]

 50%|█████     | 35204/70000 [00:01<00:01, 24093.88it/s]

 55%|█████▍    | 38159/70000 [00:01<00:01, 25683.84it/s]

 59%|█████▊    | 40971/70000 [00:01<00:01, 26398.41it/s]

 63%|██████▎   | 43872/70000 [


10500
1500


### Set up Naive Bayes Training Procedure

In [ ]:
# We will not waste 20 min figuring out slicing lists today :D
# take a list of words and get all character ngrams from the words and their counts
import collections
def get_char_ngrams(words, n=3, start_token="<s>", end_token="</s>"):
    counter = collections.defaultdict(int)
    for word in words:
        chars = [start_token] + list(word) + [end_token]
        for i in range(len(chars)-(n-1)):
            ngram = chars[i:i+n]
            counter[''.join(ngram)] += 1
    return counter

# show how it works
get_char_ngrams("she sells seashells by the seashore".lower().split())

defaultdict(int,
            {'<s>sh': 1,
             'she': 2,
             'he</s>': 2,
             '<s>se': 3,
             'sel': 1,
             'ell': 2,
             'lls': 2,
             'ls</s>': 2,
             'sea': 2,
             'eas': 2,
             'ash': 2,
             'hel': 1,
             '<s>by': 1,
             'by</s>': 1,
             '<s>th': 1,
             'the': 1,
             'sho': 1,
             'hor': 1,
             'ore': 1,
             're</s>': 1})

In [ ]:
# simple tokenization/normalization + char ngram function from above

import nltk
nltk.download('punkt')
import string

# document is a string containing 1 or more sentences
# returns a list of all of the tokens in the document
# either word-level unigrams (if char_ngrams is 0 or None) or character ngrams
def tokenize(document, char_ngrams=3):
    doc_tokens = []
    # use nltk sentence tokenization
    sentences = nltk.sent_tokenize(document)
    for sentence in sentences:
        # use nltk word tokenization
        sent_tokens = nltk.word_tokenize(sentence)
        # remove punctuation
        sent_tokens = [word.translate(str.maketrans('', '', string.punctuation)) for word in sent_tokens]
        # lowercase and remove empty strings (all punctuation will become empty after previous line)
        sent_tokens = [word.lower() for word in sent_tokens if word]
        # either use char ngrams or full words
        if char_ngrams:
            doc_tokens += get_char_ngrams(sent_tokens, char_ngrams)
        else:
            doc_tokens += sent_tokens
    return doc_tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
for x in tqdm(range(1000)):
    y = 1 + 1

100%|██████████| 1000/1000 [00:00<00:00, 1218920.08it/s]


#### Write a function to compute Naive Bayes

Need to compute 3 outputs from set of documents and their labels:
- Log-Prior: $log(\frac{|\{d \in D | C(d)=c\}|}{|D|})$
- Log-Likelihood: $log(\frac{count(w,c)+1}{\sum_{w'\in V} count(w',c)+1})$
- Vocabulary: $V$

In [ ]:
import math
import collections
from tqdm import tqdm

def train_nb(training_data, tokenize_function):

    vocab = set()
    label_counts = {}
    label_word_counts = {}
    label_sum = collections.defaultdict(int)

    num_total_docs = len(training_data['labels'])
    # Update 0: added tqdm to make a progress bar
    for d in tqdm(range(num_total_docs)):
        document = training_data['text'][d]
        label = training_data['labels'][d]

        if label not in label_counts:
            label_counts[label] = 0
        if label not in label_word_counts:
            label_word_counts[label] = collections.defaultdict(int)

        tokens = tokenize_function(document)
        for token in tokens:
            label_word_counts[label][token] += 1
            vocab.add(token)
            label_sum[label] += 1

        label_counts[label] += 1

    # keys are labels, values are the log priors
    log_priors = {}
    for label, count in label_counts.items():
        log_priors[label] = math.log(count / num_total_docs)
    # keys are labels, values are dictionaries mapping words to their 
    #                   likelihoods for that label
    log_likelihoods = {}
    for label, word_count in label_word_counts.items():
        log_likelihoods[label] = {}
        # Update 1:
        # key thing here: needed to this for all words in vocab
        # otherwise, tokens that don't appear in one of the classes
        # will not get smoothed and counts remain 0 for that class
        # then during test time, won't handle those words correctly
        # when computing the class probabilities
        # OLD VERSION WAS: 
        # for word, count in word_count.items()
        for word in vocab:
            log_likelihoods[label][word] = math.log((word_count[word] + 1) / (label_sum[label] + len(vocab)))


    return log_priors, log_likelihoods, vocab

# test sentences
train_nb({'text':['this is an okay sentence','this is a bad sentence'],'labels':['neutral','bad']}, tokenize)

100%|██████████| 2/2 [00:00<00:00, 133.70it/s]


({'neutral': -0.6931471805599453, 'bad': -0.6931471805599453},
 {'neutral': {'nce': -3.044522437723423,
   'an</s>': -3.044522437723423,
   'thi': -3.044522437723423,
   'nte': -3.044522437723423,
   'ent': -3.044522437723423,
   'bad': -3.7376696182833684,
   'sen': -3.044522437723423,
   'his': -3.044522437723423,
   'is</s>': -3.044522437723423,
   'enc': -3.044522437723423,
   '<s>a</s>': -3.7376696182833684,
   '<s>ba': -3.7376696182833684,
   'ay</s>': -3.044522437723423,
   'kay': -3.044522437723423,
   '<s>se': -3.044522437723423,
   '<s>is': -3.044522437723423,
   'ten': -3.044522437723423,
   'oka': -3.044522437723423,
   '<s>ok': -3.044522437723423,
   'ad</s>': -3.7376696182833684,
   'ce</s>': -3.044522437723423,
   '<s>an': -3.044522437723423,
   '<s>th': -3.044522437723423},
  'bad': {'nce': -2.995732273553991,
   'an</s>': -3.6888794541139363,
   'thi': -2.995732273553991,
   'nte': -2.995732273553991,
   'ent': -2.995732273553991,
   'bad': -2.995732273553991,
   'sen'

##### Dr. Wilson's example NB train function

In [ ]:
# list of tuples, document as a string and then its label

import math
import collections
from tqdm import tqdm

def my_train_nb(training_data):
    label_counts = collections.defaultdict(int)
    total_words_with_label = collections.defaultdict(int)
    word_with_label_counts = {}
    vocab = set()
    total_docs = len(training_data['text'])
    for d in tqdm(range(total_docs)):
        document = training_data['text'][d]
        label = training_data['labels'][d]
        label_counts[label] += 1
        if label not in word_with_label_counts:
            word_with_label_counts[label] = collections.defaultdict(int)
        words = tokenize(document)
        for word in words:
            vocab.add(word)
            word_with_label_counts[label][word] += 1
            total_words_with_label[label] += 1
    priors = {label:count/total_docs for label,count in label_counts.items()}
    log_likelihoods = {}
    for label, word_counts in word_with_label_counts.items():
        log_likelihoods[label] = {}
        for word in vocab:
            count = word_counts[word]
            log_likelihoods[label][word] = math.log( (count + 1) / 
                                                    (total_words_with_label[label] + len(vocab)) )
            
    return priors, log_likelihoods, vocab

my_train_nb({'text':['this is an okay sentence','this is a bad sentence'],'labels':['neutral','bad']})

100%|██████████| 2/2 [00:00<00:00, 547.56it/s]


({'neutral': 0.5, 'bad': 0.5},
 {'neutral': {'nce': -3.044522437723423,
   'an</s>': -3.044522437723423,
   'thi': -3.044522437723423,
   'nte': -3.044522437723423,
   'ent': -3.044522437723423,
   'bad': -3.7376696182833684,
   'sen': -3.044522437723423,
   'his': -3.044522437723423,
   'is</s>': -3.044522437723423,
   'enc': -3.044522437723423,
   '<s>a</s>': -3.7376696182833684,
   '<s>ba': -3.7376696182833684,
   'ay</s>': -3.044522437723423,
   'kay': -3.044522437723423,
   '<s>se': -3.044522437723423,
   '<s>is': -3.044522437723423,
   'ten': -3.044522437723423,
   'oka': -3.044522437723423,
   '<s>ok': -3.044522437723423,
   'ad</s>': -3.7376696182833684,
   'ce</s>': -3.044522437723423,
   '<s>an': -3.044522437723423,
   '<s>th': -3.044522437723423},
  'bad': {'nce': -2.995732273553991,
   'an</s>': -3.6888794541139363,
   'thi': -2.995732273553991,
   'nte': -2.995732273553991,
   'ent': -2.995732273553991,
   'bad': -2.995732273553991,
   'sen': -2.995732273553991,
   'his': 

#### Write a function to test NB on new data

In [ ]:
# take test data and the trained NB model components
# then output the test data with predicted classes
# we can use this to test on the data it was trained on 
#    and compare to the held-out test data
def test_nb(test_data, log_priors, log_likelihoods, vocabulary, tokenize_function):

    predictions = []

    for test_instance in test_data:
        document = test_instance['text']
        correct_label = test_instance['labels']

        

    # Do something here to get the predictions

##### Dr. W example solution

In [ ]:
# take test data and the trained NB model components
# then output the test data with predicted classes
# we can use this to test on the data it was trained on 
#    and compare to the held-out test data

from numpy import inf

def my_test_nb(test_data, log_priors, log_likelihoods, vocab, tokenize_function):

    predictions = []

    for d in range(len(test_data['labels'])):
        document = test_data['text'][d]
        correct_label = test_data['labels'][d]

        label_probs = collections.defaultdict(int)
        for label,log_prior in log_priors.items():
            label_probs[label] += log_prior
            tokens = tokenize_function(document)
            for token in tokens:
                label_probs[label] += log_likelihoods[label].get(token,0)
        predicted_label = None
        max_prob = -inf
        for label,prob in label_probs.items():
            if prob > max_prob:
                predicted_label = label
                max_prob = prob
        predictions.append(predicted_label)

    return predictions

In [ ]:
prior, lh, v = train_nb(small_dataset['train'], tokenize)

100%|██████████| 10500/10500 [00:10<00:00, 1048.03it/s]


In [ ]:
preds = my_test_nb(small_dataset['test'], prior, lh, v, tokenize)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(small_dataset['test']['labels'], preds))
print("Confusion Matrix: \n  |en |es |pt ")
print(confusion_matrix(small_dataset['test']['labels'], preds,labels=['en','es','pt']))

              precision    recall  f1-score   support

          en       1.00      1.00      1.00       500
          es       0.99      1.00      0.99       500
          pt       1.00      0.99      0.99       500

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500

Confusion Matrix: 
  |en |es |pt 
[[500   0   0]
 [  0 500   0]
 [  1   6 493]]


## Which features were important?

In [ ]:
# get the k features (tokens) with the highest likelihood for a given label
def get_top_k_nb_features(label, likelihoods, k):
    return sorted(likelihoods[label].items(),key=lambda x:x[1],reverse=True)[:k]

# try changing change 'es' to 'en or 'pt'
get_top_k_nb_features('pt',lh, 10)

[('<s>de', -4.665517714310733),
 ('de</s>', -4.73017111855487),
 ('<s>a</s>', -4.766357225101574),
 ('<s>um', -4.930297299982331),
 ('<s>co', -4.982536519081751),
 ('os</s>', -5.01540520936801),
 ('est', -5.056159289867391),
 ('ão</s>', -5.129223011828321),
 ('em</s>', -5.171187210927354),
 ('<s>es', -5.176917885636338)]

## What if we tried a different kind of model?

Let's train several models and compare them

In [ ]:
# model 1: character trigrams (basically, what we already did above)
trigram_tokenize = lambda document: tokenize(document, char_ngrams=3)
prior, lh, v = train_nb(small_dataset['train'], trigram_tokenize)
preds = my_test_nb(small_dataset['test'], prior, lh, v, trigram_tokenize)

# model 2: character bigrams are the tokens (aka features)
bigram_tokenize = lambda document: tokenize(document, char_ngrams=2)
prior2, lh2, v2 = train_nb(small_dataset['train'], bigram_tokenize)
preds2 = my_test_nb(small_dataset['test'], prior2, lh2, v2, bigram_tokenize)

# model 3: full words are the tokens (aka features)
full_word_tokenize = lambda document: tokenize(document, char_ngrams=None)
prior3, lh3, v3 = train_nb(small_dataset['train'], full_word_tokenize)
preds3 = my_test_nb(small_dataset['test'], prior3, lh3, v3, full_word_tokenize)

100%|██████████| 10500/10500 [00:07<00:00, 1448.36it/s]


In [ ]:
print("Model 1: Character Trigrams")
print(classification_report(small_dataset['test']['labels'], preds))
print("Model 2: Character Bigrams")
print(classification_report(small_dataset['test']['labels'], preds2))
print("Model 3: Full Word Tokens")
print(classification_report(small_dataset['test']['labels'], preds3))

Model 1: Character Trigrams
              precision    recall  f1-score   support

          en       1.00      1.00      1.00       500
          es       0.99      1.00      0.99       500
          pt       1.00      0.99      0.99       500

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500

Model 2: Character Bigrams
              precision    recall  f1-score   support

          en       1.00      1.00      1.00       500
          es       0.98      1.00      0.99       500
          pt       1.00      0.98      0.99       500

    accuracy                           0.99      1500
   macro avg       0.99      0.99      0.99      1500
weighted avg       0.99      0.99      0.99      1500

Model 3: Full Word Tokens
              precision    recall  f1-score   support

          en       1.00      1.00      1.00       500
          es       1.00      1.00      1.00     

#### That was way too easy! Let's try the full dataset

In [ ]:
# load the whole thing into memory as a dictionary
# this will make things a lot faster later since
# we will reuse this a lot and it's not _that_ big
full_dataset = {}

for split in ['train','validation','test']:
    full_dataset[split] = {'labels':[], 'text':[]}
    for item in tqdm(dataset[split]):
        lang =  item['labels']
        full_dataset[split]['labels'].append(lang)
        full_dataset[split]['text'].append(item['text'])

100%|██████████| 10000/10000 [00:00<00:00, 28893.00it/s]


In [ ]:
# model 1: character trigrams
trigram_tokenize = lambda document: tokenize(document, char_ngrams=3)
prior, lh, v = train_nb(full_dataset['train'], trigram_tokenize)
preds = my_test_nb(full_dataset['test'], prior, lh, v, trigram_tokenize)

# model 2: character bigrams are the tokens (aka features)
bigram_tokenize = lambda document: tokenize(document, char_ngrams=2)
prior2, lh2, v2 = train_nb(full_dataset['train'], bigram_tokenize)
preds2 = my_test_nb(full_dataset['test'], prior2, lh2, v2, bigram_tokenize)

# model 3: full words are the tokens (aka features)
full_word_tokenize = lambda document: tokenize(document, char_ngrams=None)
prior3, lh3, v3 = train_nb(full_dataset['train'], full_word_tokenize)
preds3 = my_test_nb(full_dataset['test'], prior3, lh3, v3, full_word_tokenize)

100%|██████████| 70000/70000 [00:21<00:00, 3198.34it/s]


In [ ]:
print("Model 1: Character Trigrams")
print(classification_report(dataset['test']['labels'], preds))
print("Model 2: Character Bigrams")
print(classification_report(dataset['test']['labels'], preds2))
print("Model 3: Full Word Tokens")
print(classification_report(dataset['test']['labels'], preds3))

Model 1: Character Trigrams
              precision    recall  f1-score   support

          ar       1.00      1.00      1.00       500
          bg       1.00      1.00      1.00       500
          de       1.00      1.00      1.00       500
          el       1.00      1.00      1.00       500
          en       0.98      1.00      0.99       500
          es       0.94      1.00      0.97       500
          fr       0.98      1.00      0.99       500
          hi       1.00      0.97      0.98       500
          it       1.00      0.97      0.99       500
          ja       1.00      1.00      1.00       500
          nl       1.00      0.99      0.99       500
          pl       1.00      0.98      0.99       500
          pt       1.00      0.95      0.97       500
          ru       1.00      1.00      1.00       500
          sw       0.94      1.00      0.97       500
          th       1.00      1.00      1.00       500
          tr       1.00      1.00      1.00       500

Let's look at some features again

In [ ]:
# what are the labels we can look at?
print(prior.keys())

dict_keys(['pt', 'bg', 'zh', 'th', 'ru', 'pl', 'ur', 'sw', 'tr', 'es', 'ar', 'it', 'hi', 'de', 'el', 'nl', 'fr', 'vi', 'en', 'ja'])


arabic (ar), bulgarian (bg), german (de), modern greek (el), english (en), spanish (es), french (fr), hindi (hi), italian (it), japanese (ja), dutch (nl), polish (pl), portuguese (pt), russian (ru), swahili (sw), thai (th), turkish (tr), urdu (ur), vietnamese (vi), and chinese (zh)

In [ ]:
get_top_k_nb_features('nl',lh,10)

[('en</s>', -5.09762314318312),
 ('an</s>', -5.708208723050045),
 ('de</s>', -5.717052747846497),
 ('een', -5.733042855087783),
 ('<s>ee', -5.768479633844694),
 ('<s>de', -5.779225203996523),
 ('et</s>', -5.83761565900736),
 ('<s>in', -5.982976639769562),
 ('in</s>', -6.012378169610847),
 ('<s>he', -6.055939063525879)]

# Hypothesis testing with bootstrap test

Let's compare 2 of our NB models: the character trigram and character bigram versions.

We know the trigram one worked better, but is this statistically significant?

*i.e., was it just better because of this specific test set we sampled? Or would we expect it to be better on the entire population this data is sampled from?*

In [ ]:
import random
from tqdm import tqdm

# take observed predictions from each model, correct labels, 
# scoring metric (function that takes correct and predictions and gives a score)
# and b, the number of bootstrap samples to take
def bootstrap_test(model_1_preds, model_2_preds, correct_labels, 
                   metric, b = 10000):

    # compute delta(x)
    observed_metric_model_1 = metric(correct_labels, model_1_preds)
    observed_metric_model_2 = metric(correct_labels, model_2_preds)
    observed_diff = observed_metric_model_1 - observed_metric_model_2

    n = len(correct_labels)
    assert len(model_1_preds) == n
    assert len(model_2_preds) == n
    s = 0

    for i in tqdm(range(b)):

        sampled_model_1_preds = []
        sampled_model_2_preds = []
        sampled_correct_labels = []

        for j in range(n):

            # choose a number 0 to n (size of test set)
            sample_idx = random.randint(0,n-1)

            # create the sample model predictions and test labels
            sampled_model_1_preds.append(model_1_preds[sample_idx])
            sampled_model_2_preds.append(model_2_preds[sample_idx])
            sampled_correct_labels.append(correct_labels[sample_idx])

        # compute delta(x_i)
        sampled_metric_model_1 = metric(sampled_correct_labels, sampled_model_1_preds)
        sampled_metric_model_2 = metric(sampled_correct_labels, sampled_model_2_preds)
        sampled_diff = sampled_metric_model_1 - sampled_metric_model_2

        # count s
        if sampled_diff >= 2*observed_diff:
            s+=1

    return s/b

In [ ]:
from sklearn.metrics import f1_score

def macro_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average="macro")

p_val = bootstrap_test(preds, preds2, dataset['test']['labels'], macro_f1)
print("p-value:",p_val)

100%|██████████| 10000/10000 [16:54<00:00,  9.86it/s]

p-value: 0.0


In [ ]:
# testing
print(bootstrap_test([0,0,0,1,1,1],[0,0,0,1,1,1],[0,0,0,1,0,0], macro_f1))
print(bootstrap_test([0,0,0,1,1,0],[0,0,0,1,1,1],[0,0,0,1,0,0], macro_f1))

100%|██████████| 10000/10000 [00:15<00:00, 659.40it/s]


1.0


100%|██████████| 10000/10000 [00:14<00:00, 690.47it/s]

0.1732
